Collaborative Filtering v_2

Collaborative filtering, ali koristeći item-user matricu. Za svakog korisnika se određuje 20% top filmova po njegovim ocjenama i preporućuju se filmovi najsličniji tom filmu koje korisnik još nije pogledao. Uzeti ćemo prvih 1000 filmova zbog vremena izvođenja.

In [3]:
import numpy as np
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity

ratings = pd.read_csv("ml-latest-small/ratings.csv")
movies = pd.read_csv("ml-latest-small/movies.csv")

df = pd.DataFrame(columns=ratings.userId.unique())

# Making item-user matrix
for index, row in movies.head(1000).groupby("movieId").count().iterrows():
    tmp = ratings.query("movieId == {}".format(index))
    mean = tmp['rating'].mean()
    for i, r in tmp.iterrows():
        df.loc[index, r['userId']] = r['rating'] - mean
df = df.fillna(0)
df

,1,2,3,4,5,6,7,8,9,10,...,601,602,603,604,605,606,607,608,609,610
1,0.079070,0.0,0.0,0.0,0.07907,0.000000,0.57907,0.000000,0.0,0.0,...,0.07907,0.000000,0.079070,-0.920930,0.079070,-1.420930,0.07907,-1.420930,-0.92093,1.079070
2,0.000000,0.0,0.0,0.0,0.00000,0.568182,0.00000,0.568182,0.0,0.0,...,0.00000,0.568182,0.000000,1.568182,0.068182,0.000000,0.00000,-1.431818,0.00000,0.000000
3,0.740385,0.0,0.0,0.0,0.00000,1.740385,0.00000,0.000000,0.0,0.0,...,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,-1.259615,0.00000,0.000000
4,0.000000,0.0,0.0,0.0,0.00000,0.642857,0.00000,0.000000,0.0,0.0,...,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.00000,0.000000
5,0.000000,0.0,0.0,0.0,0.00000,1.928571,0.00000,0.000000,0.0,0.0,...,0.00000,0.000000,0.000000,-0.071429,0.000000,0.000000,0.00000,0.000000,0.00000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1297,0.000000,0.0,0.0,0.0,0.00000,0.000000,0.00000,0.000000,0.0,0.0,...,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.00000,0.000000
1298,1.239130,0.0,0.0,0.0,0.00000,0.000000,0.00000,0.000000,0.0,0.0,...,0.00000,0.000000,-2.760870,0.000000,0.000000,0.000000,0.00000,0.000000,0.00000,0.000000
1299,0.000000,0.0,0.0,0.0,0.00000,0.000000,0.00000,0.000000,0.0,0.0,...,0.00000,0.000000,-0.017857,0.000000,0.000000,-0.517857,0.00000,0.000000,0.00000,0.000000
1300,0.000000,0.0,0.0,0.0,0.00000,0.000000,0.00000,0.000000,0.0,0.0,...,0.00000,0.000000,-1.166667,0.000000,0.000000,-0.666667,0.00000,0.000000,0.00000,0.333333


In [4]:
def findClosestMovie(n):
        movie = np.array(df.loc[n, :])
        movie = np.array([movie])
        max = 0
        movieBestId = 0
        for index, row in df.iterrows():
            if(index == n): continue
            secondMovie = np.array(row)
            secondMovie = np.array([secondMovie])
            result = cosine_similarity(movie, secondMovie)
            if(result > max):
                max = result
                movieBestId = index
        return movieBestId
    
def findRecommendationForUser(n):
    userRatings = ratings.query("userId == {} & movieId < {}".format(n, 500))

    userRatings = userRatings.sort_values('rating', ascending=False)
    topUserRatings = userRatings.head(int(0.2 * len(userRatings)))
    for index, row in topUserRatings.iterrows():
        if(len(userRatings.query("movieId == {}".format(row['movieId']))) != 0):
            indexOfClosest = findClosestMovie(row['movieId'])
            print("{} because of {}".format(movies.query("movieId == {}".format(indexOfClosest)).title,
                                            movies.query("movieId == {}".format(row['movieId'])).loc[:, 'title']))
            
findRecommendationForUser(1)

302    Ace Ventura: Pet Detective (1994)
Name: title, dtype: object because of 197    Dumb & Dumber (Dumb and Dumber) (1994)
Name: title, dtype: object
435    Mr. Wonderful (1993)
Name: title, dtype: object because of 291    Tommy Boy (1995)
Name: title, dtype: object
412    In the Line of Fire (1993)
Name: title, dtype: object because of 398    Fugitive, The (1993)
Name: title, dtype: object
828    Reservoir Dogs (1992)
Name: title, dtype: object because of 43    Seven (a.k.a. Se7en) (1995)
Name: title, dtype: object
277    Shawshank Redemption, The (1994)
Name: title, dtype: object because of 46    Usual Suspects, The (1995)
Name: title, dtype: object
